<a href="https://colab.research.google.com/github/AlfredIsair/AlfredIsair/blob/main/Clinical_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The primary objective is to classify oncology reports, with a nuanced emphasis on differentiating between colon, thyroid, and lung cancer cases. This targeted approach enables us to optimize the model for precision in handling specific oncological contexts, enhancing its effectiveness for medical text classification in the field of oncology.

## **Colab Setup**

In [1]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_8539.json to spark_nlp_for_healthcare_spark_ocr_8539.json


In [2]:
# Installing spark-nlp for GPU
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.3.0 -s $PUBLIC_VERSION -g

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

--2023-11-27 05:16:15--  https://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-11-27 05:16:16--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-11-27 05:16:16 (63.3 MB/s) - written to stdout [1191/1191]

Installi

In [3]:
!pip install -q tensorflow==2.12.0
!pip install -q tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 7.5 MB/s eta 0:00:00


In [4]:
import json
import os

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [5]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import sparknlp
import sparknlp_jsl



from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import string
import numpy as np

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"24G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params, gpu=True)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Datasets/Medical_Text_Classification.csv' ,header=None, names=["category","text"], encoding='latin1')
df.head(15)

,category,text
0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,Thyroid_Cancer,This study aimed to investigate serum matrix ...
5,Thyroid_Cancer,This study was performed to explore the effec...
6,Thyroid_Cancer,This study was performed assess the clinical ...
7,Thyroid_Cancer,Journal of International Medical Research  Th...
8,Thyroid_Cancer,Gastric cancer GC persists as a worldwide pub...
9,Thyroid_Cancer,Scars Burns HealingVolume  reuse guideli...


In [8]:
df.shape

(7570, 2)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7570 entries, 0 to 7569
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  7570 non-null   object
 1   text      7570 non-null   object
dtypes: object(2)
memory usage: 118.4+ KB


In [10]:
df.columns

Index(['category', 'text'], dtype='object')

In [11]:
df['category'].value_counts()

Thyroid_Cancer    2810
Colon_Cancer      2580
Lung_Cancer       2180
Name: category, dtype: int64

In [12]:
df.isnull().sum()

category    0
text        0
dtype: int64

**TRAIN MODEL**

In [13]:
spark_df = spark.createDataFrame(df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.7, 0.3], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1598
Test Dataset Count: 672


In [14]:
spark_df.groupBy("category").count().show()

+--------------+-----+
|      category|count|
+--------------+-----+
|Thyroid_Cancer|  827|
|   Lung_Cancer|  670|
|  Colon_Cancer|  773|
+--------------+-----+



In [15]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)



In [16]:
spark_df.head()

Row(category='Thyroid_Cancer', text="Thyroid surgery in  children in a single institution from Osama Ibrahim Almosallama Ali Aseerib Ahmed Alhumaida Ali S AlZahranic Saif Alsobhib Saud AlShanafeybFrom the  aDepartment of Surgery College of Medicine Qassim University Buraidah Al Qassim Saudi Arabia  bDepartment of Surgery King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia cDepartment of Medicine King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia Correspondence   Dr  Osama Ibrahim  Almosallam  Department of  Surgery  College  of  Medicine Qassim  University  PO  Box   Buraidah  Al  Qassim   Saudi  Arabia  osama_iaahotmailcom ORCID orcid0000000290367564        Citation  Almosallam OI Aseeri A Alhumaid A AlZahrani AS Alsobhi S AlShanafey S Thyroid surgery in  children  in  a  single  institution  from   Ann  Saudi  Med         Received January  Accepted May  Published August  Copyright Copyright   Annals of Saudi Medicine Saudi Arabia This is an   

In [17]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")


#Now we will extract healthcare_100d embeddings and use it in the classificaiton model training
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")


embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings
])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [18]:
#model.stages

In [19]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+------------+------------------------------------------------------------+
|                                                        text|    category|                                         sentence_embeddings|
+------------------------------------------------------------+------------+------------------------------------------------------------+
|    bladder cancer is the tenth most common cancer global...|Colon_Cancer|[{sentence_embeddings, 0, 32315,     bladder cancer is th...|
|    breast cancer bc is the most common malignancy among ...|Colon_Cancer|[{sentence_embeddings, 0, 32498,     breast cancer bc is ...|
+------------------------------------------------------------+------------+------------------------------------------------------------+
only showing top 2 rows



In [20]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+------------+------------------------------------------------------------+
|                                                        text|    category|                                         sentence_embeddings|
+------------------------------------------------------------+------------+------------------------------------------------------------+
|    endometrial cancer ec is a common malignancy of the f...|Colon_Cancer|[{sentence_embeddings, 0, 32555,     endometrial cancer e...|
|    the incidence and death rate of nonsmall cell lung ca...|Colon_Cancer|[{sentence_embeddings, 0, 32553,     the incidence and de...|
+------------------------------------------------------------+------------+------------------------------------------------------------+
only showing top 2 rows



In [21]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [22]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(4,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.0014882083, -0.030478254, 0.11241925, 0.16096647, -0.16610627, -0.035856325, 0.022098064, -0.047880135, 0.076399...|
|[[0.012835252, -0.009353844, 0.1010427, 0.18400054, -0.1898749, -0.03182858, 0.006118923, -0.036870006, 0.065734066, ...|
|[[0.012835252, -0.009353844, 0.1010427, 0.18400054, -0.1898749, -0.03182858, 0.006118923, -0.036870006, 0.065734066, ...|
|[[-0.04869649, -0.05703247, 0.09128266, 0.16283788, -0.19340605, -0.023796845, 0.05896229, -0.06951927, 0.023538673, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top

In [23]:
log_folder="MTC_logs_healthcare_100d"
!mkdir -p $log_folder
#this command creates a directory named "MTC_logs_healthcare_100d" in the current working directory, or it ensures that the directory exists without producing an error if it's already present.

In [24]:
classifier_dl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("category")\
    .setBatchSize(16)\
    .setMaxEpochs(30)\
    .setLr(0.002)\
    .setDropout(0.3)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath(log_folder)\
    .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
])

In [25]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings) #fitting a deep learning classifier model to the training data

In [30]:
clfDL_model_hc100.stages

[ClassifierDLModel_53df3952b706]

In [26]:
!cat $log_folder/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 1439 - classes: 3
Epoch 0/30 - 1.42s - loss: 80.500885 - acc: 0.6922285 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 1/30 - 0.87s - loss: 62.494102 - acc: 0.9212547 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 2/30 - 1.01s - loss: 57.4087 - acc: 0.9521068 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 3/30 - 0.97s - loss: 55.900723 - acc: 0.96479404 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 4/30 - 0.96s - loss: 54.759483 - acc: 0.9718633 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 5/30 - 1.09s - loss: 54.049458 - acc: 0.9781835 - batches: 90
Quality on validation dataset (10.0%), validation examples = 159
Epoch 6/30 - 1.11s - loss: 53.60502 - acc: 0.97958803 - batches: 90
Quality on validation data

In [27]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [28]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [29]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                precision    recall  f1-score   support

  Colon_Cancer       0.99      1.00      0.99       228
   Lung_Cancer       0.98      0.94      0.96       195
Thyroid_Cancer       0.96      0.99      0.97       249

      accuracy                           0.98       672
     macro avg       0.98      0.97      0.98       672
  weighted avg       0.98      0.98      0.98       672



The provided classification results indicate a high level of performance for the model, with precision, recall, and F1-score values close to 1.0 for all three classes (Colon_Cancer, Lung_Cancer, and Thyroid_Cancer). The overall accuracy is also very high at 0.97.

These results suggest that the model is able to accurately classify medical reports for all three categories. The high recall values indicate that the model is able to correctly identify a large proportion of the true positive cases. The high precision values indicate that the model is not generating many false positives.

The F1-score, which is a balance between precision and recall, is also high for all three classes. This suggests that the model is able to strike a good balance between identifying true positives and avoiding false positives.

Overall, these results are very encouraging and suggest that the model has the potential to be a valuable tool for medical diagnosis and treatment.